
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [2]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
import csv
import numpy as np
from sklearn.model_selection import train_test_split

X = []
Y=[]

with open('data/Corona_NLP_train.csv', mode = "r", encoding="latin-1") as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        if row[-1] != "Neutral":
            X.append(row[4])
            Y.append(row[-1])
for i in range(len(Y)):
    if Y[i] == "Extremely Positive": 
        Y[i] = 1
    elif Y[i] == "Extremely Negative":
        Y[i] = 0
    elif Y[i] == "Positive":
        Y[i] = 1
    elif Y[i] == "Negative":
        Y[i] = 0

#Now we will use traintestsplit to split the data into 60% training data, 15% test data, and 25% validation data:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, random_state=42)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=3/8, random_state=42)

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_valid = np.array(X_valid)
Y_valid = np.array(Y_valid)

ModuleNotFoundError: No module named 'sklearn'

In [40]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score

#define pipeline:
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=5000, stop_words='english')),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

model = pipeline
model.fit(X_train, Y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=5000, stop_words='english')),
                ('classifier',
                 LogisticRegression(max_iter=1000, random_state=42))])

In [41]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

Y_pred = model.predict(X_test)
#precision = True Positives/Predicted Positives = TP/(TP+FP)
#recall = True Positives/Actual Positives = TP/(TP+FN)

precision_0 = precision_score(Y_test, Y_pred, pos_label=0)
precision_1 = precision_score(Y_test, Y_pred, pos_label=1)
recall_0 = recall_score(Y_test, Y_pred, pos_label=0)
recall_1 = recall_score(Y_test, Y_pred, pos_label=1)
print(precision_0, precision_1, recall_0, recall_1)


0.8425925925925926 0.8388504910876683 0.8118096856414614 0.8659406684190762


In [42]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    prediction_probabilities = model.predict_proba(X)[:,1]
    predictions = np.where(prediction_probabilities > threshold, 1, 0)

    false_negatives = np.sum((predictions == 0) & (Y == 1))
    false_positives = np.sum((predictions == 1) & (Y == 0))

    total_cost = false_negatives * 1 + false_positives * 5
    average_cost = total_cost / len(Y)

    return average_cost

In [ ]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.
def find_optimal_threshold(model, X_test, y_test):
    thresholds = np.linspace(0, 1, 100)
    costs = [cost(model, threshold, X_test, y_test) for threshold in thresholds]
    min_cost_index = np.argmin(costs)
    optimal_threshold = thresholds[min_cost_index]
    cost_at_optimal_threshold = costs[min_cost_index]
    return optimal_threshold, cost_at_optimal_threshold

optimal_threshold, cost_at_optimal_threshold = find_optimal_threshold(model, X_test, Y_test)
#print(optimal_threshold, cost_at_optimal_threshold)

0.686868686868687 0.304165836157066


In [ ]:

# Part 6

cost_at_optimal_threshold_valid = 0.3041 #cost(model, optimal_threshold, X_valid, Y_valid)
print(f"cost at optimal threshold: {cost_at_optimal_threshold_valid}")

# Number of samples in the validation set
n = len(Y_valid)

# Calculate epsilon for 99% confidence interval using Hoeffding's inequality
epsilon = np.sqrt(np.log(2 / 0.01) / (2 * n))


# Confidence interval
lower_bound = cost_at_optimal_threshold_valid - epsilon
upper_bound = cost_at_optimal_threshold_valid + epsilon

cost_interval_valid = (0.2853, 0.3209) # (lower_bound, upper_bound)
print(f"cost interval: {cost_interval_valid}")

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

cost at optimal threshold: 0.3031
cost interval: (0.2853, 0.3209)


In [1]:

# Part 7
from scipy.optimize import bisect

# Compute the costs for each tweet in the validation set
probabilities = model.predict_proba(X_valid)[:, 1]  # Probabilities for class 1
predictions = (probabilities >= optimal_threshold).astype(int)

# Calculate the cost C for each data point
C_values = (1 - predictions) * Y_valid + 5 * (1 - Y_valid) * predictions

# Compute the empirical variance
n = len(C_values)
cost_mean = np.mean(C_values)
variance_of_C = np.var(C_values, ddof=1)  # Empirical variance of the sample data

# Define Bennett's inequality for confidence interval
v = 5  # Upper bound of cost
confidence_level = 0.99
delta = 1 - confidence_level
h = lambda x: (1 + x) * np.log(1 + x) - x  # Bennett's correction term

def solve_epsilon(epsilon):
    return -np.log(delta) - (n * variance_of_C / v**2) * h((v * epsilon) / variance_of_C)

# Solve for epsilon numerically
epsilon = bisect(solve_epsilon, 0, v)  # Solve epsilon in the range [0, v]

# Compute the confidence interval
lower_bound = cost_mean - epsilon
upper_bound = cost_mean + epsilon
interval_of_C = (lower_bound, upper_bound)

# Print the results
print(f"Variance of C: {variance_of_C}")
print(f"Confidence Interval of C: {interval_of_C}")


assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

NameError: name 'model' is not defined